In [28]:
import requests
import datetime
import pandas as pd
import yaml
import re
from bs4 import BeautifulSoup

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [30]:
#load dictionary to compare name between liquipedia and datdota
cmp = load_obj('lqtodat')

In [2]:
r = requests.get(url='http://liquipedia.net/dota2/api.php?action=ask&query=[[%3A%2B]][[Has+teamid%3A%3A>0]]|%3FHas+location|%3FHas+region|%3FHas+id|%3FHas+name|sort%3Dhas+id|limit%3D500&format=json')
json = r.json()

In [3]:
df = pd.DataFrame(columns=['link', 'region', 'location', 'teamlow', 'team'])
lq = []

In [4]:
for lname, ldata in json.get('query').get('results').items():
    po = ldata.get('printouts')
    tnlow = re.sub(r'\W+', '', ldata.get('fulltext')).lower()
    tn = ldata.get('fulltext')
    tl = ldata.get('fullurl')
    try:
        rg = po.get('Has region')[0]
    except IndexError:
        rg = 'Nan' 
    try:
        loc = po.get('Has location')[0]
    except IndexError:
        loc = 'Nan'        
    df = df.append({'link': tl, 'region': rg, 'location': loc, 'teamlow': tnlow, 'team': tn}, ignore_index=True)
    lq.append(tnlow)

In [5]:
urldat = 'http://datdota.com/api/teams/performances?tier=1&tier=2&valve-event=does-not-matter&threshold=5&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&patch=6.85&patch=6.84&patch=6.83&patch=6.82&patch=6.81&patch=6.80&patch=6.79&patch=6.78&patch=6.77&patch=6.76&patch=6.75&patch=6.74&winner=either&after=01%2F01%2F2011&before=20%2F02%2F2018&duration=0%3B200&duration-value-from=0&duration-value-to=200'
htmldat = requests.get(urldat)
htmldat.encoding = 'utf-8'
il = yaml.load(htmldat.text)
il = il['data']